In [41]:
# the standard imports
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Problem 2A

Data is appended as `data_problem2.csv`.

Load the data and report general information of the data.

Additionally plot (as histograms) the data and discuss the separability.

In [42]:
# ignore the headers and transpose the data
# this gives us 3600 samples and 2 features
# col 1 is a class label (0 or 1), and col 0 is the actual feature (some value)
data = pd.read_csv("data_problem2.csv", header=None)
data = data.T
# call on the methods below to get a better understanding of the data
# data.head()
# data.info()
# data.describe()

In [ ]:
label = data.iloc[:, 1]
feature = data.iloc[:, 0]
# count occurrences of class 0 and 1
class_0_count = (data == 0).sum().sum()
class_1_count = (data == 1).sum().sum()
print(f"Class 0 count:          {class_0_count}")
print(f"Class 1 count:          {class_1_count}")
print(f"Label missing values:   {label.isnull().sum()}")
print(f"Feature missing values: {feature.isnull().sum()}")
# plot the histogram and scatter plot
fig, axs = plt.subplots(1, 2, figsize=(10, 12))
axs[0].bar(["Class 0", "Class 1"], [class_0_count, class_1_count], color=["blue", "green"])
axs[0].set_title("Histogram of classes 0 and 1 for all samples")
axs[0].set_xlabel("Class")
axs[0].set_ylabel("Frequency")
axs[1].scatter(label, feature, color="black")
axs[1].set_title("Scatter plot")
axs[1].set_xlabel("Class 0 or 1")
axs[1].set_ylabel("Feature value")
plt.show()

### Problem 2C

Split the data into training and test data.

Use the maximum likelihood estimations to estimate the parameters based on the training data.

Use the point-estimations of the parameters to implement a Bayes’ classifier.

Report the test accuracy.

In [ ]:
# do the 80/20 split
class_0 = data[data.iloc[:, 1] == 0]
class_1 = data[data.iloc[:, 1] == 1]
# extract pseudo-randomly 80% to use for training
training_class_0 = class_0.sample(frac=0.8, random_state=random.randint(1, 100))
training_class_1 = class_1.sample(frac=0.8, random_state=random.randint(1, 100))
# extract pseudo-randomly 20% to use for testing
testing_class_0 = class_0.drop(training_class_0.index)
testing_class_1 = class_1.drop(training_class_1.index)
# merge the sets back together, and reset the sample index
training_set = pd.concat([training_class_0, training_class_1]).reset_index(drop=True)
testing_set = pd.concat([testing_class_0, testing_class_1]).reset_index(drop=True)
# list the stats
print(f"Number of samples in data set:        {data.shape[0]}")
print(f"80% of data set:                      {int(data.shape[0] * 0.8)}")
print(f"20% of data set:                      {int(data.shape[0] * 0.2)}")
print(f"Number of samples in training set:    {training_set.shape[0]}")
print(f"Number of samples in testing set:     {testing_set.shape[0]}")
print(f"Class 0 percentage of dataset:        {round(data[data.iloc[:, 1] == 0].shape[0] / data.shape[0] * 100, 2)}")
print(f"Class 1 percentage of dataset:        {round(data[data.iloc[:, 1] == 1].shape[0] / data.shape[0] * 100, 2)}")
print(f"Class 0 percentage of training set:   {round(training_set[training_set.iloc[:, 1] == 0].shape[0] / training_set.shape[0] * 100, 2)}")
print(f"Class 1 percentage of training set:   {round(training_set[training_set.iloc[:, 1] == 1].shape[0] / training_set.shape[0] * 100, 2)}")
print(f"Class 0 percentage of testing set:    {round(testing_set[testing_set.iloc[:, 1] == 0].shape[0] / testing_set.shape[0] * 100, 2)}")
print(f"Class 1 percentage of testing set:    {round(testing_set[testing_set.iloc[:, 1] == 1].shape[0] / testing_set.shape[0] * 100, 2)}")

In [45]:
# estimate the MLE's for the training set
# class 0 (Gamma distribution given alpha=2)
alpha = 2
train_c0_set = training_set[training_set.iloc[:, 1] == 0]
train_c0_feature = train_c0_set.iloc[:, 0]
n0 = train_c0_set.shape[0]
# class 1 (Gaussian distribution)
train_c1_set = training_set[training_set.iloc[:, 1] == 1]
train_c1_feature = train_c1_set.iloc[:, 0]
n1 = train_c1_set.shape[0]
# param MLE's
beta_hat_c0 = np.sum(train_c0_feature) / (n0 * alpha)
mu_hat_c1 = np.sum(train_c1_feature) / n1
sigma2_hat_c1 = np.sum((train_c1_feature - mu_hat_c1) ** 2) / n1
# fetch the testing set n's
n0_test = testing_set[testing_set.iloc[:, 1] == 0].shape[0]
n1_test = testing_set[testing_set.iloc[:, 1] == 1].shape[0]

In [46]:
# use the point-estimations of the parameters to implement a Bayes’ classifier
def bayes_classifier(x, alpha, beta_hat_0, mu_hat_1, sigma2_hat_1, p_c0, p_c1):
    sigma_hat_1 = np.sqrt(sigma2_hat_1)
    # p(x|C0) - Gamma distribution (likelihood)
    p_x_c0 = (1 / (beta_hat_0 ** alpha)) * (x ** (alpha - 1)) * np.exp(-x / beta_hat_0)
    # p(x|C1) - Gaussian distribution (likelihood)
    p_x_c1 = (1 / (sigma_hat_1 * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu_hat_1) / sigma_hat_1) ** 2)
    # (evidence)
    p_x = p_x_c0 * p_c0 + p_x_c1 * p_c1
    # (posteriors)
    p_c0_x = (p_x_c0 * p_c0) / p_x
    p_c1_x = (p_x_c1 * p_c1) / p_x
    # classify
    return 0 if p_c0_x > p_c1_x else 1

training_labels = training_set.iloc[:, 1]
training_features = training_set.iloc[:, 0]
# (priors)
p_c0_train = n0 / (n0 + n1)
p_c1_train = n1 / (n0 + n1)
# functional style, map the bayes classifier to each sample
predictions_on_train = training_features.map(lambda x: bayes_classifier(x, alpha, beta_hat_c0, mu_hat_c1, sigma2_hat_c1, p_c0_train, p_c1_train))

testing_labels = testing_set.iloc[:, 1]
testing_features = testing_set.iloc[:, 0]
# (priors)
p_c0_test = n0_test / (n0_test + n1_test)
p_c1_test = n1_test / (n0_test + n1_test)
# functional style, map the bayes classifier to each sample
predictions_on_test = testing_features.map(lambda x: bayes_classifier(x, alpha, beta_hat_c0, mu_hat_c1, sigma2_hat_c1, p_c0_test, p_c1_test))

In [ ]:
# and report the accuracy of the classifier on the training set
def evaluate_classification(predictions, labels):
    correct_predictions = predictions == labels # element-wise comparison
    accuracy = np.mean(correct_predictions) * 100
    tp = ((predictions == 1) & (labels == 1)).sum()
    tn = ((predictions == 0) & (labels == 0)).sum()
    fp = ((predictions == 1) & (labels == 0)).sum()
    fn = ((predictions == 0) & (labels == 1)).sum()
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    print(f"Accuracy:          {accuracy:.2f}%")
    print(f"Precision:         {precision:.2f}")
    print(f"Recall:            {recall:.2f}")
    print(f"F1 Score:          {f1_score:.2f}")
    print(f"Correctly classified Points: {(correct_predictions).sum()}")
    print(f"Misclassified Points:        {(~correct_predictions).sum()}")

evaluate_classification(predictions_on_train, training_labels)
evaluate_classification(predictions_on_test, testing_labels)

### Problem 2D

Explain why the Bayes’ classifier minimizes the probability of miss-classification when the probability distribution of the data is known.

Show the misclassified data in a plot along with the rest of the data and explain why it was misclassified.

Does it follow the conclusion in task 2A?

In [ ]:
def visualize_classification(predictions, labels, features):
    correct = predictions == labels # element-wise comparison
    plt.figure(figsize=(10, 12))
    # correctly classified points
    plt.scatter(labels[correct], features[correct], color="green", label="Correctly classified")
    # misclassified points
    plt.scatter(labels[~correct], features[~correct], color="red", label="Misclassified")
    plt.xlabel("Label")
    plt.ylabel("Feature")
    plt.title("Classification results")
    plt.legend()
    plt.show()

visualize_classification(predictions_on_train, training_labels, training_features)
visualize_classification(predictions_on_test, testing_labels, testing_features)